In [ ]:
import pandas as pd
import numpy as np

In [ ]:
passangers = pd.read_csv(
    "../../data/Пассажирообороты  метрополитена усреднённые.csv",
    encoding="cp1251",
    delimiter=";",
    engine="python"
)

In [ ]:
subways = pd.read_csv(
    "../../data/subway_entrances.csv",
    delimiter="\t",
    engine="python",
    index_col=["Name"]
)
subways = subways.drop('Unnamed: 0', axis=1)

In [ ]:
new_data = pd.DataFrame(
    {"Название ОП": [], "Геопозиция": [], "Пассажирооборот": []})

for i in range(0, passangers.shape[0], 4):
    weight = (passangers.iloc[i, 6:].sum()*5+passangers.iloc[i+1, 6:].sum()*2)/7
    weight += (passangers.iloc[i+2, 6:].sum()*5+passangers.iloc[i+1+2, 6:].sum()*2)/7
    row = [passangers.iloc[i, 1], None, weight]
    new_data = pd.concat(
        [pd.DataFrame([row], columns=new_data.columns), new_data], ignore_index=True)

new_data

In [ ]:
for i in [1, 2]:
    zenit1 = new_data.loc[new_data["Название ОП"] == f"Зенит-{i}/1"]
    zenit2 = new_data.loc[new_data["Название ОП"] == f"Зенит-{i}/2"]
    row = [f"Зенит - {i}", None,
            zenit1.iloc[0, 2] + zenit2.iloc[0, 2]]
    new_data = pd.concat(
        [pd.DataFrame([row], columns=new_data.columns), new_data], ignore_index=True)
    
    new_data = new_data.drop(
        new_data.index[new_data["Название ОП"] == f"Зенит-{i}/1"])
    new_data = new_data.drop(
        new_data.index[new_data["Название ОП"] == f"Зенит-{i}/2"])

new_data

In [ ]:
kupchino2 = new_data.loc[new_data["Название ОП"] == "Купчино - 2"]
ladoga1 = new_data.loc[new_data["Название ОП"] == "Ладожская 1"]
new_data.loc[new_data["Название ОП"] ==
                "Ладожская"].iloc[0, 2] += ladoga1.iloc[0, 2]
new_data.loc[new_data["Название ОП"] ==
                "Купчино"].iloc[0, 2] += kupchino2.iloc[0, 2]
new_data = new_data.drop(
    new_data.index[new_data["Название ОП"] == "Ладожская 1"])
new_data = new_data.drop(
    new_data.index[new_data["Название ОП"] == "Купчино - 2"])

In [ ]:
new_data

In [ ]:
subways

In [ ]:
for i in range(new_data.shape[0]):
    name = new_data.iloc[i, 0]
    location = subways.loc[name]
    new_data.iloc[i, 1] = location
new_data = new_data.set_index("Название ОП")

new_data

In [ ]:
new_data.to_csv("../../data/result/subway_passangers_traffic.csv", sep=",", encoding="utf-8")